<a href="https://colab.research.google.com/github/leeje008/ML_final_project/blob/main/tree/main/final_code/(4)Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Using Colab

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import  confusion_matrix,recall_score,precision_score,accuracy_score
from imblearn.metrics import geometric_mean_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE




def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    g_mean = geometric_mean_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:4f}, 정밀도 {1:4f}, 재현율 {2:4f}, AUC {3:4f},G_MEAN {4:4f}'.format(accuracy , precision , recall,roc,g_mean))
    

def get_model_train_eval(model,ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)



import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Loan_Train.csv')
df.head()

df = df.drop('Id',axis = 1)


from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

data = df


for e in data.columns:
    if data[e].dtype == 'object':
        labelEncoder.fit(list(data[e].values))
        data[e] = labelEncoder.transform(data[e].values)
        
        # Accommodate the data that has been changed
        df = data
        
y = df.Risk_Flag
X = df.drop('Risk_Flag', axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   random_state=101)



# Hyperpameter tunning RF

In [ ]:
! pip install smote-variants

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
import smote_variants as sv
from sklearn.metrics import roc_auc_score, make_scorer

     |████████████████████████████████| 134 kB 8.1 MB/s 
  Created wheel for minisom: filename=MiniSom-2.2.9-py3-none-any.whl size=8594 sha256=6c4ab293d4a0a97fed650932956fd37c297e52e05b71121c8d3e9a9f6428b2cd
  Stored in directory: /root/.cache/pip/wheels/3d/a1/10/f50b6f4865652eac239a2700de411c3078c27e1318320e494c
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7452 sha256=5ee91de6d04ba1dea8322a442cfcebe66206f218674a698ebc4e15616d8a944e
  Stored in directory: /root/.cache/pip/wheels/37/09/e1/49ee45c0ce248a6e9c986aae82d32bbcba09c9f3b9d0c3406a
Successfully built minisom statistics


In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SMOTE", SMOTE(random_state = 101)),
                            ("RandomForestClassifier", RandomForestClassifier())
                            ])

param_grid = {
    "SMOTE__sampling_strategy": [0.5,0.75,1],
    "RandomForestClassifier__n_estimators": [50,75,100,125,150]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model
smote_best_model = gs_pipeline.best_estimator_

y_validation_preds = smote_best_model.predict(X_test)
roc_auc_score(y_test, y_validation_preds)


scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/RF_SMOTE_1.csv',index = False)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.5; total time=  15.4s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.5; total time=  15.4s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.5; total time=  15.3s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.5; total time=  15.5s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.5; total time=  15.4s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.75; total time=  19.0s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.75; total time=  18.7s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.75; total time=  18.6s
[CV] END RandomForestClassifier__n_estimators=50, SMOTE__sampling_strategy=0.75; total time=  19.2s
[CV] END RandomForestClassifier__n_estimator

In [ ]:
roc_auc_score(y_test, y_validation_preds)

0.8544567223273958

# RF ADA

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("ada", ADASYN(random_state = 101)),
                            ("RandomForestClassifier", RandomForestClassifier())
                            ])

param_grid = {
    "ada__sampling_strategy": [0.5,0.75,1],
    "RandomForestClassifier__n_estimators": [50,75,100,125,150]
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model


scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/RF_ADA.csv',index = False)


smote_best_model = gs_pipeline_1.best_estimator_

y_validation_preds = smote_best_model.predict(X_test)
roc_auc_score(y_test, y_validation_preds)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.5; total time=  14.9s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.5; total time=  14.6s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.5; total time=  14.6s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.5; total time=  14.5s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.5; total time=  14.5s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.75; total time=  17.6s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.75; total time=  17.2s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.75; total time=  17.3s
[CV] END RandomForestClassifier__n_estimators=50, ada__sampling_strategy=0.75; total time=  16.9s
[CV] END RandomForestClassifier__n_estimators=50, ada__samplin

0.8577438357846991

In [ ]:
roc_auc_score(y_test, y_validation_preds)

0.8577438357846991

# RF SVR

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SVR", smote_variants.NDO_sampling()),
                            ("RF", RandomForestClassifier())
                            ])

param_grid = {
    "SVR__proportion": [0.5,0.75,1],
    "RF__n_estimators": [50,75,100,125,150]
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(np.array(X_train), np.array(y_train))

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model


scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/RF_SVD.csv',index = False)

svr_best_model = gs_pipeline_1.best_estimator_


y_validation_preds = svr_best_model.predict(np.array(X_test))

roc_auc_score(y_test, y_validation_preds)

# LGB hyperparameter tunning
# LGB SMOTE

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [(("SMOTE",  SMOTE(random_state = 101))),
                             ("LGBClassifier", LGBMClassifier(boost_from_average = False))
                             ])

param_grid = {
    "SMOTE__sampling_strategy": [0.5,0.75,1],
    "LGBClassifier__n_estimators": [100,200,300,400,500]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")



scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/LGB_SMOTE.csv',index = False)



svr_best_model = gs_pipeline.best_estimator_


y_validation_preds = svr_best_model.predict(np.array(X_test))

roc_auc_score(y_test, y_validation_preds)

In [ ]:
X_sm, y_sm = SMOTE(random_state = 101).fit_resample(X_train, y_train)
clf =   LGBMClassifier(boost_from_average = False)

clf.fit(X_sm, y_sm)
pred = clf.predict(X_test)

roc_auc_score(y_test,pred)

0.7483717646590253

# LGB ADASYN


In [ ]:
start = time.time()


pipeline = Pipeline(steps= [(("ada",  ADASYN(random_state = 101))),
                             ("LGBClassifier", LGBMClassifier(boost_from_average = False))
                             ])

param_grid = {
    "ada__sampling_strategy": [0.5,0.75,1],
    "LGBClassifier__n_estimators": [100,200,300,400,500]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")



scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/LGB_ADASYN_1.csv',index = False)

# LGB SVR

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SVR", sv.NDO_sampling()),
                            ("LGB", LGBMClassifier())
                            ])

param_grid = {
    "SVR__proportion": [0.5,0.75,1],
    "LGB__n_estimators": [100,200,300,400,500]
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(np.array(X_train), np.array(y_train))

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model


scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/LGB_SVR.csv',index = False)

svr_best_model = gs_pipeline_1.best_estimator_


y_validation_preds = svr_best_model.predict(np.array(X_test))

roc_auc_score(y_test, y_validation_preds)

# KNN Tunning

# KNN SMOTE

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SMOTE", SMOTE(random_state = 101)),
                            ("KNN", KNeighborsClassifier())
                            ])

param_grid = {
    "SMOTE__sampling_strategy": [0.5,0.75,1],
    "KNN__n_neighbors": list(range(10,101,10))
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")
# 결과 저장
scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/KNN_SMOTE.csv',index = False)

# Store the best model


smote_best_model = gs_pipeline.best_estimator_

y_validation_preds = smote_best_model.predict(X_test)
roc_auc_score(y_test, y_validation_preds)

# KNN ADA

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("ADASYN", ADASYN(random_state = 101)),
                            ("KNN", KNeighborsClassifier())
                            ])

param_grid = {
    "ADASYN__sampling_strategy": [0.5,0.75,1],
    "KNN__n_neighbors": list(range(10,101,10))
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")

scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/KNN_ADA.csv',index = False)

# Store the best model
ada_best_model = gs_pipeline_1.best_estimator_

y_validation_preds = ada_best_model.predict(X_test)
roc_auc_score(y_test, y_validation_preds)

scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)

# KNN SVR

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SVR", sv.distance_SMOTE()),
                            ("KNeighborsClassifier", KNeighborsClassifier())
                            ])

param_grid = {
    "SVR__proportion": [0.5,0.75,1],
    "KNeighborsClassifier__n_neighbors": list(range(10,101,10))
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(np.array(X_train), np.array(y_train))

end = time.time()


scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/KNN_SVR.csv',index = False)

print(f"{end - start:.5f} sec")

svr_best_model = gs_pipeline_1.best_estimator_


y_validation_preds = svr_best_model.predict(np.array(X_test))

roc_auc_score(y_test, y_validation_preds)